In [5]:
%load_ext autoreload
%autoreload 2

from pymongo import MongoClient
from bson.objectid import ObjectId

mongo_client = MongoClient()

database = mongo_client["graduation"]

pull_requests_collection = database["pull_requests"]

projects_collection = database["projects"]

commits_collection = database["commits"]

import analysis_utilities

#'_id' : {'$lt' : ObjectId("5bca53e00000000000000000")}

projects = list(projects_collection.find({'succeeded' : True, 'travis_is_oldest_ci': True, 'scrape_type': {'$in': ['travis_1', 'random_1']}}))

print("Found {} projects with a travis build status date".format(len(projects)))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 685 projects with a travis build status date


In [12]:
enough_prs = 0
did = 0

prs_before_eff = []
prs_after_eff = []

for project in projects:
        prs = list(pull_requests_collection.find({'project_name': project["full_name"].split("/")[1],'project_owner': project["full_name"].split("/")[0]},
                                                {'created_at': 1, 'review_comments': 1}))

        prs_before, prs_after = analysis_utilities.split_prs_on_build_date(project, prs, True, "status_travis_date")
       
        prs_before_eff.extend([pr for pr in prs_before if any([comment for comment in pr['review_comments'] if "is_effective" in comment and comment["is_effective"]])])
        prs_after_eff.extend([pr for pr in prs_after if any([comment for comment in pr['review_comments'] if "is_effective" in comment and comment["is_effective"]])])
        
            
        did += 1
        
        if did % 100 == 0:
            print("Did {} projects".format(did))
            
print("Found {} projects with enough prs".format(enough_prs))

Did 100 projects
Did 200 projects
Did 300 projects
Did 400 projects
Did 500 projects
Did 600 projects
Found 0 projects with enough prs


In [24]:
effective_comments_before = [comment for pr in prs_before_eff for comment in pr["review_comments"] if "is_effective" in comment and comment["is_effective"]]
effective_comments_after = [comment for pr in prs_after_eff for comment in pr["review_comments"] if "is_effective" in comment and comment["is_effective"]]

print("There are {} eff comments before and {} eff comments after".format(len(effective_comments_before), len(effective_comments_after)))

import random

random.seed(210)

sample_size = 30

print("--------------- before")
for comment in random.sample(effective_comments_before, sample_size):
    print(comment["url"])
    
    
print("--------------- after")
for comment in random.sample(effective_comments_after, sample_size):
    print(comment["url"])

There are 46065 eff comments before and 57030 eff comments after
--------------- before
https://api.github.com/repos/doctrine/doctrine2/pulls/comments/359260
https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/13546877
https://api.github.com/repos/yast/yast-network/pulls/comments/18271677
https://api.github.com/repos/elastic/logstash/pulls/comments/161666
https://api.github.com/repos/caskdata/cdap/pulls/comments/40270786
https://api.github.com/repos/symfony-cmf/symfony-cmf-docs/pulls/comments/6668032
https://api.github.com/repos/Smoothieware/Smoothieware/pulls/comments/18431695
https://api.github.com/repos/mopidy/mopidy/pulls/comments/75616
https://api.github.com/repos/ipa320/cob_robots/pulls/comments/6910649
https://api.github.com/repos/cfpb/cfgov-refresh/pulls/comments/18413987
https://api.github.com/repos/AnalyticalGraphicsInc/cesium/pulls/comments/3625055
https://api.github.com/repos/openemr/openemr/pulls/comments/72394684
https://api.github.com/repos/adobe/brackets/pu